<a href="https://colab.research.google.com/github/Chirag314/EDA/blob/main/Multistep_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import packages
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import time
import os

pd.set_option('display.float_format',lambda x: '%.4f' %x)
import seaborn as sns 
sns.set_context('paper',font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller

from keras.layers import *
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# Example of multi step forecasting time series.

# Generate dummy data

def generate_time_series(batch_size, n_steps):
  freq1,freq2,offsets1,offsets2=np.random.rand(4,batch_size,1)
  time = np.linspace(0,1,n_steps)
  series = 0.5 * np.sin((time-offsets1)*(freq1*10+10))
  series  = series + 0.2*np.sin((time-offsets2)*(freq2*20+20))
  series  = series + 0.1 * (np.random.rand(batch_size, n_steps)-0.5)
  return series[...,np.newaxis].astype(np.float32)

In [3]:
# Define parameters
n_steps = 50
series = generate_time_series(10000,n_steps+1)
x_train, y_train = series [:7000,:n_steps], series[:7000,-1]
x_valid, y_valid= series[7000:9000,:n_steps],series[7000:9000,-1]
x_test, y_test = series[9000:,:n_steps],series[9000:,-1]
y_train.shape

(7000, 1)

In [4]:
# Baseline metric
y_pred =x_valid[:,-1]
np.mean(keras.losses.mean_squared_error(y_valid,y_pred))

0.020596031

In [5]:
# USe fully connected network and check error
model= keras.models.Sequential([
                                keras.layers.Flatten(input_shape=[50,1]),
                                keras.layers.Dense(1)
                      
])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss='mae',optimizer=optimizer,metrics='accuracy')

model.fit(x_train,y_train,epochs=20,validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0818 - accuracy: 0.0000e+00 - val_loss: 0.0659 - val_accuracy: 0.0000e+00
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0571 - accuracy: 0.0000e+00 - val_loss: 0.0566 - val_accuracy: 0.0000e+00
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0559 - accuracy: 0.0000e+00 - val_loss: 0.0538 - val_accuracy: 0.0000e+00
Epoch 4/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0534 - accuracy: 0.0000e+00 - val_loss: 0.0497 - val_accuracy: 0.0000e+00
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0527 - accuracy: 0.0000e+00 - val_loss: 0.0475 - val_accuracy: 0.0000e+00
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0531 - accuracy: 0.0000e+00 - val_loss: 0.0491 - val_accuracy: 0.0000e+00
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0518 - accuracy: 0.0000e

In [6]:
# Use simple RNN

model1= keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(1)
])

model.compile(optimizer=optimizer, loss='mse', metrics='accuracy')

model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0032 - accuracy: 0.0000e+00 - val_loss: 0.0031 - val_accuracy: 0.0000e+00
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 0.0000e+00 - val_loss: 0.0030 - val_accuracy: 0.0000e+00
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 0.0000e+00 - val_loss: 0.0031 - val_accuracy: 0.0000e+00
Epoch 4/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 0.0000e+00 - val_loss: 0.0031 - val_accuracy: 0.0000e+00
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 0.0000e+00 - val_loss: 0.0031 - val_accuracy: 0.0000e+00
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 0.0000e+00 - val_loss: 0.0031 - val_accuracy: 0.0000e+00
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 0.0000e

In [7]:
# Forecasting several time steps ahead.
series = generate_time_series(10000,n_steps+10)
x_train, y_train = series[:7000,:n_steps],series[:7000,-10:,0]
x_valid, y_valid =series[7000:9000,:n_steps],series[7000:9000,-10:,0]
x_test, y_test= series[9000:, :n_steps], series[9000:,-10,0]

In [8]:
model2= keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(10)
])

model2.compile(loss='mse', optimizer=optimizer)

model2.fit(x_train,y_train, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 7s 25ms/step - loss: 0.0626 - val_loss: 0.0451
Epoch 2/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0396 - val_loss: 0.0365
Epoch 3/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0358 - val_loss: 0.0335
Epoch 4/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0306 - val_loss: 0.0242
Epoch 5/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0285 - val_loss: 0.0409
Epoch 6/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0294 - val_loss: 0.0315
Epoch 7/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0840 - val_loss: 0.1039
Epoch 8/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0720 - val_loss: 0.0499
Epoch 9/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0523 - val_loss: 0.0475
Epoch 10/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0490 - val_lo

In [9]:

# Add time distributed layer
model3= keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[50,1]),
                                 keras.layers.SimpleRNN(20,return_sequences=True),
                                 keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model3.compile(loss='mse', optimizer=optimizer)

model3.fit(x_train,y_train, epochs=20)

Epoch 1/20


InvalidArgumentError: ignored

In [ ]:
y_train.shape

In [ ]:
# Add time distributed layer
model4= keras.models.Sequential([
                                 keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.LSTM(20,return_sequences=True),
                                 keras.layers.Dense(10)
])

model4.compile(loss='mse', optimizer=optimizer)

model4.fit(x_train,y_train, epochs=20)